<img src="https://www.th-koeln.de/img/logo.svg" style="float:right;" width="200">

# 12th exercise: <font color="#C70039">First Reinforcement Learning Game (*Frozen Lake*) using OpenAI Gym</font>
* Course: AML
* Lecturer: <a href="https://www.gernotheisenberg.de/">Gernot Heisenberg</a>
* Author of notebook: Mathis Loevenich
* Date:   27.12.2022
* Matr. No.: 11157363 

<img src="https://launchyourintelligentapphome.files.wordpress.com/2019/05/frozenlake_legended.png?w=531" style="float: center;" width="600">

---------------------------------
**GENERAL NOTE 1**: 
Please make sure you are reading the entire notebook, since it contains a lot of information on your tasks (e.g. regarding the set of certain paramaters or a specific computational trick), and the written mark downs as well as comments contain a lot of information on how things work together as a whole. 

**GENERAL NOTE 2**: 
* Please, when commenting source code, just use English language only. 
* When describing an observation please use English language, too.
* This applies to all exercises throughout this course.

---------------------------------

### <font color="ce33ff">DESCRIPTION</font>:

#### OpenAI Gym
In this exercise you will be using Python and OpenAI Gym to develop your reinforcement learning algorithm. The Gym library is a collection of environments that can be used freely with the reinforcement learning algorithms.

Gym has a ton of environments ranging from simple text based games to Atari games like Breakout and Space Invaders. The library is intuitive to use and simple to install. Just run **pip install gym** and you are good to go! The link to Gym's installation instructions, requirements, and documentation is included in the description. 

Further reading about OpenAI Gym is available under https://www.gymlibrary.dev/.
This notebook is based on this great post and notebook from [Rodolfo Mendes](https://morioh.com/p/18a96b9091d3).

#### Frozen Lake
This description of the game is copied directly from Gym's website. 

*Winter is coming. You and your friends were tossing around a frisbee at the park when you made a wild throw that left the frisbee out in the middle of the lake. The water is mostly frozen, but there are a few holes where the ice has melted. If you step into one of those holes, you'll fall into the freezing water and die (Game over). At this time, there's an international frisbee shortage, so it's absolutely imperative that you navigate across the lake and retrieve the disc. However, the ice is slippery, so you won't always move in the direction you intend. The surface is described using a grid like the following:*

* SFFF
* FHFH
* FFFH
* HFFG

This grid is your environment! S is your (the agent's) starting point and it's safe. F represents the frozen surface and is also safe. H represents a hole and if your agent steps in a hole in the middle of a frozen lake, the game is over because the agent dies. Finally, G represents the goal, which is the space on the grid where the frisbee is located.

The agent can navigate *left, right, up, down* and the episode ends when the agent reaches the goal or falls in a hole. It receives a reward of **1** if it reaches the goal and **0** otherwise.

Here is the summary:
<img src="./images/FrozenLake.States.Rewards.png" style="float: center;" width="800">

---------------------------------

### <font color="FFC300">TASKS</font>:
The tasks that you need to work on within this notebook are always indicated below as bullet points. 
If a task is more challenging and consists of several steps, this is indicated as well. 
Make sure you have worked down the task list and commented your doings. 
This should be done by using markdown.<br> 
<font color=red>Make sure you don't forget to specify your name and your matriculation number in the notebook.</font>

**YOUR TASKS in this exercise are as follows**:
1. import the notebook to Google Colab or use your local machine.
2. make sure you specified you name and your matriculation number in the header below my name and date. 
    * set the date too and remove mine.
3. read the entire notebook carefully 
    * add comments whereever you feel it necessary for better understanding
    * run the notebook for the first time. 
4. install gym into your env!
5. You will train an agent to play the *Frozen Lake* game using Q-learning and you will get a playback of how the agent does after being trained.
6. Again the task: Your agent has to navigate the grid by staying on the frozen surface without falling into any holes until it reaches the frisbee. If it reaches the frisbee, it wins with a reward of plus one. If it falls in a hole, it loses and receives no points for the entire episode.
7. Your tasks are highlighted in the notebook (see below)
---------------------------------

### Imports 
import all important libs including gym

In [1]:
import numpy as np
import gym
import random
import time
from   IPython.display import clear_output

### Creating the Environment
For creating your environment, just call *gym.make()* and pass a string of the name of the environment you want to set up. 
All the environments with their corresponding names you can use here are available on Gym's website (see above).
With this *env* object, you are able to query for information about the environment, sample states and actions, retrieve rewards and have your agent navigate the frozen lake. That is all made available to you conveniently with Gym.

In [2]:
env = gym.make("FrozenLake-v1")

/usr/local/lib/python3.8/dist-packages/gym/core.py:317: DeprecationWarning: WARN: Initializing wrapper in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(
/usr/local/lib/python3.8/dist-packages/gym/wrappers/step_api_compatibility.py:39: DeprecationWarning: WARN: Initializing environment in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(


### Creating the Q-Table
Now, construct your Q-table, and initialize all the Q-values to zero for each state-action pair.
The number of rows in the table is equivalent to the size of the state space in the environment, and the number of columns is equivalent to the size of the action space (see above). You can get this information using *env.observation_space.n* and *env.action_space.n* as shown below in the code. Then, you can use this information to build the Q-table and initialize it with zeros.

In [3]:
action_space_size = env.action_space.n
state_space_size = env.observation_space.n

q_table = np.zeros((state_space_size, action_space_size))

In [4]:
print(q_table)

[[0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]]


### Initializing Q-Learning hyperparameters
Now, we're going to create and initialize all the parameters needed to implement the Q-learning algorithm.

First, with *num_episodes*, you define the total number of episodes you want the agent to play during training. Then, with *max_steps_per_episode*, you define a maximum number of steps that your agent is allowed to take within a single episode. So, if by the 100th step, the agent has not reached the frisbee or fallen through a hole, then the episode will terminate with the agent receiving zero points.

Next, you will set your *learning_rate* and your *discount_rate* as well, which was represented with the symbol (lambda) in the course slides (keyword: discounted return G_t).

Now, the last four parameters are all related to the exploration-exploitation dilemma with respect to the epsilon-greedy policy. You are initializing your *exploration_rate* to **1** and setting the *max_exploration_rate* to **1** and a *min_exploration_rate* to **0.01**. The *max* and *min* are just bounds to how large or small your exploration rate can be. Remember, the exploration rate was represented with the symbol (epsilon) when discussed in the course slides.

Lastly, you will set the *exploration_decay_rate* to **0.01** to determine the rate at which the *exploration_rate* will decay.

**YOUR <font color="FFC300">TASK</font> in this exercise is as follows** (point 7 from the task list above):

All of the above parameters can change!
Your task is to create a *testplan* and tune all parameters by yourself and observe how they influence and change the performance of the algorithm. 
Make notes! They will help you during the exam.

In [5]:
num_episodes = 10000
max_steps_per_episode = 100

learning_rate = 0.1
discount_rate = 0.99

exploration_rate = 1
max_exploration_rate = 1
min_exploration_rate = 0.01
exploration_decay_rate = 0.01

Create a list to hold all of the rewards you will get from each episode. 
By means of this you can observe how your game score changes over time.

In [6]:
rewards_all_episodes = []

In the following code section, the entire Q-learning algorithm is implemented as discussed in detail in the AML course. 
When this code is executed, this is exactly where the training will take place. 
* The first for-loop contains everything that happens within a single episode. 
* The second nested loop contains everything that happens for a single time-step.

Read all the red comments, as they contain lots of important information on the implementation.

In [54]:
# Q-learning algorithm

def qlearning(
    num_episodes = 10000, 
    max_steps_per_episode = 100, 
    learning_rate = 0.1,
    discount_rate = 0.99,
    exploration_rate = 1,
    max_exploration_rate = 1,
    min_exploration_rate = 0.01,
    exploration_decay_rate = 0.01
    ):
  
  action_space_size = env.action_space.n
  state_space_size = env.observation_space.n

  q_table = np.zeros((state_space_size, action_space_size))
  
  rewards_all = []

  # loop: for a single episode
  for episode in range(num_episodes):
      # initialize 'new episode' parameters
      state = env.reset()
      ''' The done variable just keeps track of whether or not your episode is finished.
      Initialize it to False when first starting the episode and you will see later where 
      it will get updated to notify you when the episode is over.'''
      done = False
      
      ''' Keep track of the rewards within the current episode as well.
      Hence, set rewards_current_episode = 0 since you start 
      with no rewards at the beginning of each episode.'''
      rewards_current_episode = 0

      # nested loop: for a single time-step
      for step in range(max_steps_per_episode): 
          # Exploration-exploitation trade-off
          '''For each time-step within an episode set your exploration_rate_threshold 
          to a random number between 0 and 1. This will be used to determine whether 
          your agent will explore or exploit the environment in this time-step.'''
          exploration_rate_threshold = random.uniform(0, 1)
          if exploration_rate_threshold > exploration_rate:
              action = np.argmax(q_table[state,:]) 
          else:
              action = env.action_space.sample()

          # Take new action
          '''After action is chosen, take that action by calling step() on your env object and 
          pass your action to it. The function step() returns a tuple containing the new state, 
          the reward for the action you took, whether or not the action ended the episode and 
          diagnostic information regarding the environment (helpful for debugging).'''
          new_state, reward, done, info = env.step(action)

          # Update Q-table for Q(s,a)
          '''Compare this implementation with the equation in the course slides.'''
          q_table[state, action] = q_table[state, action] * (1 - learning_rate) + \
          learning_rate * (reward + discount_rate * np.max(q_table[new_state, :]))
          
          '''Set your current state to the new_state that was returned when taking the last action 
          and then update the rewards from your current episode by adding the reward you received 
          for your previous action.'''
          # Set new state
          state = new_state
          # Add new reward 
          rewards_current_episode += reward 
          '''Then, check to see if your last action ended the episode 
          (game over by agent stepping in a hole or reaching the goal)! 
          If the action did end the episode, then jump out of this loop and start the next episode.
          Otherwise, transition to the next time-step.'''
          if done == True: 
              break
              

      # Exploration rate decay
      '''Once an episode is finished, you need to update your exploration_rate using exponential decay, 
      which just means that the exploration rate decays at a rate proportional to its current value. 
      You can decay the exploration_rate using the formula above, which makes use of all the exploration 
      rate parameters that were defined above in the hyperparameter section.'''
      exploration_rate = min_exploration_rate + (max_exploration_rate - min_exploration_rate) * np.exp(-exploration_decay_rate*episode)
      
      # Add current episode reward to total rewards list and move on to the next episode
      rewards_all.append(rewards_current_episode)

  return rewards_all


### All episodes training completed
After all episodes are finished you now just calculate the average reward per thousand episodes from your list that contains the rewards for all episodes so that you can print it out and see how the rewards changed over time.

In [24]:
# Calculate and print the average reward per thousand episodes
num_episodes=10000
rewards_all_episodes = qlearning(num_episodes=num_episodes)
rewards_per_thousand_episodes = np.split(np.array(rewards_all_episodes),num_episodes/1000)
count = 1000

print("********Average reward per thousand episodes********\n")
for r in rewards_per_thousand_episodes:
    print(count, ": ", str(sum(r/1000)))
    count += 1000

********Average reward per thousand episodes********

1000 :  0.07400000000000005
2000 :  0.08200000000000006
3000 :  0.08400000000000006
4000 :  0.09600000000000007
5000 :  0.1350000000000001
6000 :  0.1320000000000001
7000 :  0.3000000000000002
8000 :  0.4210000000000003
9000 :  0.6710000000000005
10000 :  0.6750000000000005


### Interpretation

From this print, you can see that the average reward per thousand episodes did indeed progress over time. When the algorithm first started training, the first thousand episodes only averaged a reward of almost **0.18**, but by the time it got to its last thousand episodes, the reward drastically improved to almost **0.7**.

Let's take a second to understand how you can interpret these results. Your agent played **10000** episodes. At each time step within an episode, the agent received a reward of **1** if it reached the frisbee, otherwise, it received a reward of **0**. If the agent did indeed reach the frisbee, then the episode finished at that time-step.

Hence, that means for each episode, the total reward received by the agent for the entire episode is either **1** or **0**. So, for the first thousand episodes, you can interpret this score as meaning that **18%** of the time the agent received a reward of **1** and won the episode. And by the last thousand episodes from a total of **10000**, the agent was winning almost **70%** of the time.

By analyzing the grid of the game, you can see it is a lot more likely that the agent would fall in a hole or perhaps reach the max time steps than it is to reach the frisbee, so reaching the frisbee **70%** of the time is not too bad, especially since the agent had no explicit instructions to reach the frisbee. It learned that this is the correct thing to do.

* SFFF
* FHFH
* FFFH
* HFFG

At last, print out your updated Q-table to see how that has transitioned from its initial state of all zeros.

# Exploration

Recap about the metrics:

- Learning rate: This parameter controls the step size at which the agent updates its policy. A higher learning rate will cause the agent to update its policy more quickly, but it may also cause the agent to overshoot its optimal policy.

- Discount rate: This parameter controls the agent's preference for immediate rewards versus future rewards. A higher discount rate will cause the agent to focus more on immediate rewards, while a lower discount rate will cause the agent to focus more on long-term rewards.

- Exploration: This parameter controls the agent's willingness to try out new actions, even if they have not been proven to be rewarding in the past. A higher exploration rate will cause the agent to try out new actions more often, while a lower exploration rate will cause the agent to rely more on actions that have been proven to be rewarding in the past.


In [10]:
from ipywidgets import interact, fixed

In [55]:
def get_rewards(
    num_episodes = 10000, 
    learning_rate = 0.1,
    discount_rate = 0.99,
    exploration_rate = 1,
    exploration_decay_rate = 0.01
):
    # Calculate and print the average reward per thousand episodes
  rewards_all_episodes = qlearning(
      num_episodes = num_episodes, 
      learning_rate = learning_rate,
      discount_rate = discount_rate,
      exploration_rate = exploration_rate,
      exploration_decay_rate = exploration_decay_rate
      )
  rewards_per_thousand_episodes = np.split(np.array(rewards_all_episodes),num_episodes/1000)
  count = 1000

  print("********Average reward per thousand episodes********\n")
  for r in rewards_per_thousand_episodes:
      print(count, ": ", str(sum(r/1000)))
      count += 1000

In [12]:
interact(
    get_rewards,
    num_episodes = (1000, 10000, 1000),
    learning_rate = (0,1,0.01),
    discount_rate = (0, 1, 0.01),
    exploration_rate = (0, 1, 0.01),
    exploration_decay_rate = fixed(0.001)
    )

interactive(children=(IntSlider(value=10000, description='num_episodes', max=10000, min=1000, step=1000), Floa…

<function __main__.get_rewards(num_episodes=10000, learning_rate=0.1, discount_rate=0.99, exploration_rate=1, exploration_decay_rate=0.01)>

In [56]:
# Calculate and print the average reward per thousand episodes
get_rewards(
    num_episodes=10000,
    learning_rate = 0.1,
    discount_rate = .99,
    exploration_rate = 1,
    exploration_decay_rate = 0.01
    )

********Average reward per thousand episodes********

1000 :  0.1290000000000001
2000 :  0.19900000000000015
3000 :  0.4870000000000004
4000 :  0.7040000000000005
5000 :  0.6810000000000005
6000 :  0.6780000000000005
7000 :  0.6680000000000005
8000 :  0.6470000000000005
9000 :  0.6800000000000005
10000 :  0.6920000000000005


In [57]:
# Calculate and print the average reward per thousand episodes
get_rewards(
    num_episodes=10000,
    learning_rate = 0.2,
    discount_rate = .99,
    exploration_rate = 1,
    exploration_decay_rate = 0.01
    )

********Average reward per thousand episodes********

1000 :  0.44100000000000034
2000 :  0.6510000000000005
3000 :  0.6560000000000005
4000 :  0.6710000000000005
5000 :  0.6540000000000005
6000 :  0.6650000000000005
7000 :  0.6590000000000005
8000 :  0.6550000000000005
9000 :  0.7010000000000005
10000 :  0.6570000000000005


In [62]:
get_rewards(
    num_episodes=10000,
    learning_rate = 0.05,
    discount_rate = .99,
    exploration_rate = 1,
    exploration_decay_rate = 0.01
    )

********Average reward per thousand episodes********

1000 :  0.06700000000000005
2000 :  0.07800000000000006
3000 :  0.08100000000000006
4000 :  0.21200000000000016
5000 :  0.4170000000000003
6000 :  0.3860000000000003
7000 :  0.36900000000000027
8000 :  0.3820000000000003
9000 :  0.4820000000000004
10000 :  0.5550000000000004


In [87]:
get_rewards(
    num_episodes=10000,
    learning_rate = 0.2,
    discount_rate = .98,
    exploration_rate = 1,
    exploration_decay_rate = 0.01
    )

********Average reward per thousand episodes********

1000 :  0.2860000000000002
2000 :  0.5800000000000004
3000 :  0.6750000000000005
4000 :  0.6330000000000005
5000 :  0.6650000000000005
6000 :  0.6540000000000005
7000 :  0.6560000000000005
8000 :  0.6400000000000005
9000 :  0.6680000000000005
10000 :  0.6430000000000005


### Key Findings:

- If we set a higher learning_rate between 0.1 and 0.2 we can a better result in the first thousand epochs but we get worse results in the long run.

- If we decrease the learning rate, our results get worse

- There is no real better improvement to set the learning rate higher than 2.

- slightly decreasing the discount_rate by 1 % makes the output worse

- If the exploration decay ray is too high it happens often that the reward is always 0



In [89]:
get_rewards(
    num_episodes=10000,
    learning_rate = 0.2,
    discount_rate = 0.99,
    exploration_rate = 1,
    exploration_decay_rate = 0.001
    )

********Average reward per thousand episodes********

1000 :  0.05100000000000004
2000 :  0.19100000000000014
3000 :  0.3830000000000003
4000 :  0.5290000000000004
5000 :  0.6160000000000004
6000 :  0.6240000000000004
7000 :  0.6710000000000005
8000 :  0.6690000000000005
9000 :  0.6330000000000005
10000 :  0.6530000000000005


In [101]:
get_rewards(
    num_episodes=20000,
    learning_rate = 0.05,
    discount_rate = 0.99,
    exploration_rate = 1,
    exploration_decay_rate = 0.001
    )

********Average reward per thousand episodes********

1000 :  0.04000000000000003
2000 :  0.21300000000000016
3000 :  0.4200000000000003
4000 :  0.5720000000000004
5000 :  0.6390000000000005
6000 :  0.6730000000000005
7000 :  0.6730000000000005
8000 :  0.6780000000000005
9000 :  0.6660000000000005
10000 :  0.6990000000000005
11000 :  0.6670000000000005
12000 :  0.6830000000000005
13000 :  0.6930000000000005
14000 :  0.6880000000000005
15000 :  0.6700000000000005
16000 :  0.6760000000000005
17000 :  0.6750000000000005
18000 :  0.6820000000000005
19000 :  0.7090000000000005
20000 :  0.6900000000000005


In [114]:
get_rewards(
    num_episodes=20000,
    learning_rate = 0.1,
    discount_rate = 0.995,
    exploration_rate = 1,
    exploration_decay_rate = 0.001
    )

********Average reward per thousand episodes********

1000 :  0.05600000000000004
2000 :  0.21300000000000016
3000 :  0.4020000000000003
4000 :  0.5530000000000004
5000 :  0.6300000000000004
6000 :  0.6840000000000005
7000 :  0.6550000000000005
8000 :  0.6870000000000005
9000 :  0.6500000000000005
10000 :  0.6640000000000005
11000 :  0.6680000000000005
12000 :  0.6820000000000005
13000 :  0.6710000000000005
14000 :  0.6750000000000005
15000 :  0.6880000000000005
16000 :  0.6680000000000005
17000 :  0.6930000000000005
18000 :  0.6920000000000005
19000 :  0.6710000000000005
20000 :  0.6960000000000005


### Key Findings with num_episodes=20000

- I found out that with lower eploration_decay_rate and learning_rate the model gets better over longer time periods but performs bad at the start.
- In combination with a slightly higher discount_rate (because immidiate results are always better in this kind of setup) and a learning rate lower between 0.05 and 0.2 it's mostly always the case to get close to 70%

In [115]:
get_rewards(
    num_episodes=50000,
    learning_rate = 0.05,
    discount_rate = 0.995,
    exploration_rate = 1,
    exploration_decay_rate = 0.0005
    )

********Average reward per thousand episodes********

1000 :  0.023000000000000013
2000 :  0.06700000000000005
3000 :  0.1550000000000001
4000 :  0.2850000000000002
5000 :  0.3770000000000003
6000 :  0.44900000000000034
7000 :  0.5670000000000004
8000 :  0.5570000000000004
9000 :  0.6130000000000004
10000 :  0.6310000000000004
11000 :  0.6460000000000005
12000 :  0.6720000000000005
13000 :  0.6860000000000005
14000 :  0.6770000000000005
15000 :  0.7130000000000005
16000 :  0.6870000000000005
17000 :  0.6880000000000005
18000 :  0.6810000000000005
19000 :  0.7050000000000005
20000 :  0.6560000000000005
21000 :  0.6890000000000005
22000 :  0.6860000000000005
23000 :  0.6760000000000005
24000 :  0.6980000000000005
25000 :  0.6760000000000005
26000 :  0.6540000000000005
27000 :  0.6920000000000005
28000 :  0.6910000000000005
29000 :  0.6960000000000005
30000 :  0.7090000000000005
31000 :  0.6880000000000005
32000 :  0.7030000000000005
33000 :  0.6790000000000005
34000 :  0.6860000000000005

In [120]:
get_rewards(
    num_episodes=50000,
    learning_rate = 0.1,
    discount_rate = 0.995,
    exploration_rate = 1,
    exploration_decay_rate = 0.0005
    )

********Average reward per thousand episodes********

1000 :  0.026000000000000016
2000 :  0.07000000000000005
3000 :  0.1430000000000001
4000 :  0.2460000000000002
5000 :  0.35900000000000026
6000 :  0.45300000000000035
7000 :  0.5430000000000004
8000 :  0.5610000000000004
9000 :  0.5750000000000004
10000 :  0.6440000000000005
11000 :  0.6550000000000005
12000 :  0.6380000000000005
13000 :  0.6870000000000005
14000 :  0.6540000000000005
15000 :  0.6900000000000005
16000 :  0.6610000000000005
17000 :  0.6970000000000005
18000 :  0.6670000000000005
19000 :  0.6470000000000005
20000 :  0.6790000000000005
21000 :  0.6870000000000005
22000 :  0.6820000000000005
23000 :  0.6780000000000005
24000 :  0.7050000000000005
25000 :  0.6890000000000005
26000 :  0.6500000000000005
27000 :  0.6650000000000005
28000 :  0.6510000000000005
29000 :  0.6790000000000005
30000 :  0.6850000000000005
31000 :  0.6790000000000005
32000 :  0.6880000000000005
33000 :  0.6830000000000005
34000 :  0.667000000000000